# LDA - GBT Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score


from datasets import load_dataset

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = load_dataset("hate_speech_filipino")

In [4]:
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

X_train, Y_train = train_set["text"], train_set["label"]
X_val, Y_val = validation_set["text"], validation_set["label"]
X_test, Y_test = test_set["text"], test_set["label"]

X = X_train + X_val + X_test
Y = Y_train + Y_val + Y_test

In [5]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [6]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)

## Perform Latent Dirichlet Allocation on the training set

In [7]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics


In [ ]:
# Transform validation and test data using the fitted vectorizer and LDA
X_val_counts = vectorizer.transform(X_val)
X_val_topics = lda.transform(X_val_counts)

X_test_counts = vectorizer.transform(X_test)
X_test_topics = lda.transform(X_test_counts)

## Train the Gradient-boosting Tree

In [ ]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2)
        max_depth = trial.suggest_int("max_depth", 10, 100)
        gbt = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=RANDOM_SEED)
        gbt.fit(X_train, Y_train)
        Y_pred = gbt.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [ ]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_GBT_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

In [ ]:
best_n_estimators = study.best_trial.params["n_estimators"]
best_learning_rate = study.best_trial.params["learning_rate"]
best_max_depth = study.best_trial.params["max_depth"]

gbt = GradientBoostingClassifier(n_estimators=best_n_estimators, 
                                 learning_rate=best_learning_rate,
                                 max_depth=best_max_depth,
                                 random_state=RANDOM_SEED)
gbt.fit(X_train_topics, Y_train)
Y_pred = gbt.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")